In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import warnings

warnings.filterwarnings('ignore')

np.random.seed(42)

In [ ]:
df = pd.read_csv("/Users/zafiraibraeva/Code/uni coding/thesis/thesis_code/thesis/webapp/dataset/final_data.csv")

print("Dataset Info:")
df.info()

df.head()

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
features = [
    "Station1_CO", "Station1_NO2", "Station1_NOx",
    "Station2_CO", "Station2_NO2", "Station2_NOx", "Station2_O3",
    "Station1_SO2", "Station2_SO2",
    "Station1_PM10", "temp", "humidity", "precip",
    "precipcover", "cloudcover", "windspeed", "visibility",
    "winddir_sin", "winddir_cos", "is_heating_season", "is_work_day",
    "year", "month", "day"
]
target = "Station2_PM10"

print("\nMissing values:")
print(df[features + [target]].isnull().sum())

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df[target], kde=True)
plt.title('PM10 Distribution at Station 2')
plt.xlabel('PM10 Concentration')
plt.ylabel('Frequency')
plt.show()

In [ ]:
X = df[features].values
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


def create_sequences(data, targets, time_steps=30):
    X_seq, y_seq = [], []
    for i in range(time_steps, len(data)):
        X_seq.append(data[i - time_steps:i, :])
        y_seq.append(targets[i])
    return np.array(X_seq), np.array(y_seq)

time_steps = 30
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, time_steps)

print(f"Training data shape: {X_train_scaled.shape}")
print(f"Test data shape: {X_test_scaled.shape}")
print(f"Training sequences shape: {X_train_seq.shape}")
print(f"Test sequences shape: {X_test_seq.shape}")

In [ ]:
print("Training GBDT model...")
gbdt = GradientBoostingRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    min_samples_split=5,
    random_state=42,
    verbose=1
)
gbdt.fit(X_train_scaled, y_train)

In [ ]:
gbdt_pred = gbdt.predict(X_test_scaled)
gbdt_mse = mean_squared_error(y_test, gbdt_pred)
gbdt_mae = mean_absolute_error(y_test, gbdt_pred)
gbdt_r2 = r2_score(y_test, gbdt_pred)

print(f"\nGBDT Performance:")
print(f"MSE: {gbdt_mse:.4f}")
print(f"MAE: {gbdt_mae:.4f}")
print(f"R2: {gbdt_r2:.4f}")

plt.figure(figsize=(12, 6))
feat_importances = pd.Series(gbdt.feature_importances_, index=features)
feat_importances.nlargest(15).plot(kind='barh')
plt.title('GBDT Feature Importance')
plt.show()

In [ ]:
print("Training DART model...")
dart = LGBMRegressor(
    boosting_type='dart',
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    drop_rate=0.1,
    random_state=42,
    verbose=-1
)
dart.fit(X_train_scaled, y_train)

In [ ]:
dart_pred = dart.predict(X_test_scaled)
dart_mse = mean_squared_error(y_test, dart_pred)
dart_mae = mean_absolute_error(y_test, dart_pred)
dart_r2 = r2_score(y_test, dart_pred)

print(f"\nDART Performance:")
print(f"MSE: {dart_mse:.4f}")
print(f"MAE: {dart_mae:.4f}")
print(f"R2: {dart_r2:.4f}")

plt.figure(figsize=(12, 6))
feat_importances = pd.Series(dart.feature_importances_, index=features)
feat_importances.nlargest(15).plot(kind='barh')
plt.title('DART Feature Importance')
plt.show()

In [ ]:
print("Training LightGBM model...")
lgbm = LGBMRegressor(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)
lgbm.fit(X_train_scaled, y_train)

In [ ]:
lgbm_pred = lgbm.predict(X_test_scaled)
lgbm_mse = mean_squared_error(y_test, lgbm_pred)
lgbm_mae = mean_absolute_error(y_test, lgbm_pred)
lgbm_r2 = r2_score(y_test, lgbm_pred)

print(f"\nLightGBM Performance:")
print(f"MSE: {lgbm_mse:.4f}")
print(f"MAE: {lgbm_mae:.4f}")
print(f"R2: {lgbm_r2:.4f}")

plt.figure(figsize=(12, 6))
feat_importances = pd.Series(lgbm.feature_importances_, index=features)
feat_importances.nlargest(15).plot(kind='barh')
plt.title('LightGBM Feature Importance')
plt.show()

In [ ]:
lstm_model = Sequential([
    LSTM(512, return_sequences=True, input_shape=(time_steps, X_train_seq.shape[2])),
    Dropout(0.3),
    LSTM(256),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer=Adam(learning_rate=0.001),
                   loss='mse',
                   metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("Training LSTM model...")
lstm_history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(lstm_history.history['loss'], label='Train Loss')
plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training History')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
lstm_pred = lstm_model.predict(X_test_seq).flatten()
lstm_mse = mean_squared_error(y_test_seq, lstm_pred)
lstm_mae = mean_absolute_error(y_test_seq, lstm_pred)
lstm_r2 = r2_score(y_test_seq, lstm_pred)

print(f"\nLSTM Performance:")
print(f"MSE: {lstm_mse:.4f}")
print(f"MAE: {lstm_mae:.4f}")
print(f"R2: {lstm_r2:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(y_test_seq[:200], label='Actual PM10', alpha=0.7)
plt.plot(lstm_pred[:200], label='Predicted PM10', alpha=0.7)
plt.title('LSTM Predictions vs Actual (First 200 Samples)')
plt.xlabel('Samples')
plt.ylabel('PM10 Concentration')
plt.legend()
plt.show()

In [ ]:
gru_model = Sequential([
    GRU(128, return_sequences=True, input_shape=(time_steps, X_train_seq.shape[2])),
    Dropout(0.3),
    GRU(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

gru_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

print("Training GRU model...")
gru_history = gru_model.fit(
    X_train_seq, y_train_seq,
    epochs=150,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(gru_history.history['loss'], label='Train Loss')
plt.plot(gru_history.history['val_loss'], label='Validation Loss')
plt.title('GRU Training History')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
gru_pred = gru_model.predict(X_test_seq).flatten()
gru_mse = mean_squared_error(y_test_seq, gru_pred)
gru_mae = mean_absolute_error(y_test_seq, gru_pred)
gru_r2 = r2_score(y_test_seq, gru_pred)

print(f"\nGRU Performance:")
print(f"MSE: {gru_mse:.4f}")
print(f"MAE: {gru_mae:.4f}")
print(f"R2: {gru_r2:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(y_test_seq[:200], label='Actual PM10', alpha=0.7)
plt.plot(gru_pred[:200], label='Predicted PM10', alpha=0.7)
plt.title('GRU Predictions vs Actual (First 200 Samples)')
plt.xlabel('Samples')
plt.ylabel('PM10 Concentration')
plt.legend()
plt.show()

In [ ]:
def get_meta_features(X_tree, X_seq, models):
    tree_preds = {
        'gbdt': models['gbdt'].predict(X_tree),
        'dart': models['dart'].predict(X_tree),
        'lgbm': models['lgbm'].predict(X_tree)
    }

    seq_preds = {
        'lstm': models['lstm'].predict(X_seq).flatten(),
        'gru': models['gru'].predict(X_seq).flatten()
    }

    # Align predictions
    min_len = min(len(p) for p in [*tree_preds.values(), *seq_preds.values()])

    meta_X = np.column_stack([
        tree_preds['gbdt'][-min_len:],
        tree_preds['dart'][-min_len:],
        tree_preds['lgbm'][-min_len:],
        seq_preds['lstm'][-min_len:],
        seq_preds['gru'][-min_len:]
    ])

    return meta_X

In [ ]:
meta_X_train = get_meta_features(X_train_scaled[time_steps:], X_train_seq,
                                 {'gbdt': gbdt, 'dart': dart, 'lgbm': lgbm,
                                  'lstm': lstm_model, 'gru': gru_model})
meta_y_train = y_train[time_steps:len(X_train_scaled[time_steps:]) + time_steps]

meta_X_test = get_meta_features(X_test_scaled[time_steps:], X_test_seq,
                                {'gbdt': gbdt, 'dart': dart, 'lgbm': lgbm,
                                 'lstm': lstm_model, 'gru': gru_model})
meta_y_test = y_test[time_steps:len(X_test_scaled[time_steps:]) + time_steps]

print(f"Meta-features training shape: {meta_X_train.shape}")
print(f"Meta-features test shape: {meta_X_test.shape}")

In [ ]:
meta_model = Sequential([
    Dense(64, activation='relu', input_dim=meta_X_train.shape[1]),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

meta_model.compile(optimizer=Adam(learning_rate=0.001),
                   loss='mse')

print("Training meta-model...")
meta_history = meta_model.fit(
    meta_X_train, meta_y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(meta_history.history['loss'], label='Train Loss')
plt.plot(meta_history.history['val_loss'], label='Validation Loss')
plt.title('Meta-Model Training History')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
base_results = {
    'GBDT': {'mse': gbdt_mse, 'mae': gbdt_mae, 'r2': gbdt_r2},
    'DART': {'mse': dart_mse, 'mae': dart_mae, 'r2': dart_r2},
    'LightGBM': {'mse': lgbm_mse, 'mae': lgbm_mae, 'r2': lgbm_r2},
    'LSTM': {'mse': lstm_mse, 'mae': lstm_mae, 'r2': lstm_r2},
    'GRU': {'mse': gru_mse, 'mae': gru_mae, 'r2': gru_r2}
}

print("Base Model Performance:")
display(pd.DataFrame(base_results).T)

In [ ]:
ensemble_pred = meta_model.predict(meta_X_test).flatten()
ensemble_mse = mean_squared_error(meta_y_test, ensemble_pred)
ensemble_mae = mean_absolute_error(meta_y_test, ensemble_pred)
ensemble_r2 = r2_score(meta_y_test, ensemble_pred)

print("\nEnsemble Model Performance:")
print(f"MSE: {ensemble_mse:.4f}")
print(f"MAE: {ensemble_mae:.4f}")
print(f"R2: {ensemble_r2:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(meta_y_test[:200], label='Actual PM10', alpha=0.7)
plt.plot(ensemble_pred[:200], label='Predicted PM10', alpha=0.7)
plt.title('Ensemble Predictions vs Actual (First 200 Samples)')
plt.xlabel('Samples')
plt.ylabel('PM10 Concentration')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(meta_y_test, label='Actual PM10')
plt.plot(ensemble_pred, label='Predicted PM10', alpha=0.7, marker='.')
plt.title('Ensemble Predictions vs Actual')
plt.xlabel('Samples')
plt.ylabel('PM10 Concentration')
plt.legend()
plt.show()

In [ ]:
meta_model.compile(optimizer='adam', loss='mse')  
meta_model.save('ensemble_model.keras', include_optimizer=True)  